In [2]:
function solve_quadratic1(a,b,c)
    d = sqrt(b^2 - 4*a*c)
    (-b + d)/(2*a), (-b - d)/(2*a)
end   

solve_quadratic1 (generic function with 1 method)

In [3]:
r1 = 8.0e7

8.0e7

In [4]:
r2 = 1/r1

1.25e-8

Expanding $(x - r_1) (x - r_2) = x^2 - (r_1+r_2) x + r_1 r_2$, we see the coefficients are

In [6]:
(a,b,c) = (1,-r1-r2,r1*r2)

(1, -8.000000000000001e7, 1.0)

The solutions are

In [8]:
solve_quadratic1(a,b,c)

(8.0e7, 1.4901161193847656e-8)

One solution is $8.00 \times 10^7$ as it should be, but the other solution should be  $1.25 \times 10^{-8}$, but we get $1.4901161193847656 \times 10^{-8}$. Oh, that's _terrible_.  What has happended?  Apparently, our calculation has been corrupted by subtractive cancellation. When $b > 0$ and $b^2 \gg 4 a c$, we have $\sqrt{b^2 - 4 a c} \approx b$. And that makes the addition in $-b + \sqrt{b^2 - 4 a c}$ ill conditioned.

Fortunately, there is a cure. There is a lesser-known variant of the quadratic formula that puts the radical in the demonominator. Specifically,
$$
 -b \pm \sqrt{d}  =  -b \pm \sqrt{d} \times \frac{ -b \mp \sqrt{d}}{ -b \mp \sqrt{d}} = \frac{b^2  -  d}{ -b \mp \sqrt{d}}
   = \frac{4 a c}{ -b \mp \sqrt{d}}.
$$


Using the alternative quadratic formula with the radicals in the denominator, our function is

In [10]:
function solve_quadratic2(a,b,c)
    d = sqrt(b^2 - 4*a*c)
    2*c/(-b - d), 2*c/(-b + d)
end   

solve_quadratic2 (generic function with 1 method)

This time we get an accurate value for $1.25 \times 10^{-8}$, but not for the root $8.0 \times 10^{7}$. This is very perverse-both methods give _one_ inaccurate result

In [12]:
solve_quadratic2(a,b,c)

(6.7108864e7, 1.25e-8)

The problem is with an ill-conditioned sum $-b + d$ or $-b - d$. Since $d$ is positive (it's the output of a square root), the sum
$-b + d$ is possibly ill-conditioned when $b > 0$. And when $b < 0$, the sum $-b - d$ is possibly ill-conditioned. 

To simplify the logic, let's negate all the coefficients when $b < 0$. After that we know that $b$ is positive, so the sum $-b - d$ is well-conditioned. This gives

In [14]:
function solve_quadratic3(a,b,c)
    if b < 0 (a,b,c) = (-a,-b,-c) end #now b > 0
    d = sqrt(b^2 - 4*a*c)
    (-b - d)/(2*a), 2*c/(-b-d) # all sums well conditioned
end   

solve_quadratic3 (generic function with 1 method)

Now both roots are accurate!

In [16]:
 solve_quadratic3(a,b,c)

(8.0e7, 1.25e-8)

Should we declare victory? No way. Our function `solve_quadratic3` can suffer from subtractive cancellation when it computes the discriminant $b^2 - 4ac$.  Here is an example that is due to William Kahn (https://people.eecs.berkeley.edu/%7Ewkahan/Qdrtcs.pdf).

In [18]:
(a,b,c) = (94906265.625, -2.0 * 94906267.000,94906268.375)

(9.4906265625e7, -1.89812534e8, 9.4906268375e7)

In [19]:
b*b - 4*a*c

0.0

So `solve_quadratic3` gives two roots that are nearly identical

In [21]:
 (r1,r2) = solve_quadratic3(a,b,c)

(1.0000000144879793, 1.000000014487979)

Let's convert $a,b$ and $c$ to big floats and re-compute the roots:

In [23]:
(aa,bb,cc) = map(x -> convert(BigFloat,x), (a,b,c));

In [24]:
(rr1, rr2) = solve_quadratic3(aa,bb,cc)

(1.000000028975958351011137469355042911583320450556396022983861872923629745862732, 1.0)

According to this, only about 8 of 16 digits of the roots computed using `Float64` numbers are correct

In [26]:
r1-rr1

-1.448797909027560902762695018153800184704077102298386187292362974586273239852133e-08

In [27]:
r2-rr2

1.44879790386909235166967846453189849853515625e-08

Is there a cure? Well, yes, one cure is to convert to numbers of the type `BigFloat.` Is there a cure that only uses numbers of the type `Float64?`  Yes there is--it involves a tricky use of the `fma` function. The method can be found in the article by Claude-Pierre Jeannerod, Nicolas Louvet, and Jean-Michel Muller (https://hal.inria.fr/ensl-00649347/en). But we won't go into this method.

The quadratic formula is a very famous example of an ill-conditioned sum (aka subtractive cancellation); for references, see, for example

 -  https://en.wikipedia.org wiki/Quadratic_equation#Avoiding_loss_of_significance. 

-  http://i.stanford.edu/pub/cstr/reports/cs/tr/66/40/CS-TR-66-40.pdf


As we have observed, there are additional problems when computing $b^2 - 4ac$ is ill-conditioned.

What about solving a cubic?  The solutions to $x^3 + b x^2 + c x + d = 0$ are

$$\left[ x=\left({{-1}\over{2}}-{{\sqrt{3}\,i}\over{2}}\right)\,
 \left({{\sqrt{27\,d^2+\left(4\,b^3-18\,b\,c\right)\,d+4\,c^3-b^2\,c^
 2}}\over{2\,3^{{{3}\over{2}}}}}+{{b\,c-3\,d}\over{6}}+{{\left(-1
 \right)\,b^3}\over{27}}\right)^{{{1}\over{3}}}-{{\left({{\sqrt{3}\,i
 }\over{2}}+{{-1}\over{2}}\right)\,\left({{c}\over{3}}+{{\left(-1
 \right)\,b^2}\over{9}}\right)}\over{\left({{\sqrt{27\,d^2+\left(4\,b
 ^3-18\,b\,c\right)\,d+4\,c^3-b^2\,c^2}}\over{2\,3^{{{3}\over{2}}}}}+
 {{b\,c-3\,d}\over{6}}+{{\left(-1\right)\,b^3}\over{27}}\right)^{{{1
 }\over{3}}}}}+{{\left(-1\right)\,b}\over{3}} , x=\left({{\sqrt{3}\,i
 }\over{2}}+{{-1}\over{2}}\right)\,\left({{\sqrt{27\,d^2+\left(4\,b^3
 -18\,b\,c\right)\,d+4\,c^3-b^2\,c^2}}\over{2\,3^{{{3}\over{2}}}}}+{{
 b\,c-3\,d}\over{6}}+{{\left(-1\right)\,b^3}\over{27}}\right)^{{{1
 }\over{3}}}-{{\left({{-1}\over{2}}-{{\sqrt{3}\,i}\over{2}}\right)\,
 \left({{c}\over{3}}+{{\left(-1\right)\,b^2}\over{9}}\right)}\over{
 \left({{\sqrt{27\,d^2+\left(4\,b^3-18\,b\,c\right)\,d+4\,c^3-b^2\,c^
 2}}\over{2\,3^{{{3}\over{2}}}}}+{{b\,c-3\,d}\over{6}}+{{\left(-1
 \right)\,b^3}\over{27}}\right)^{{{1}\over{3}}}}}+{{\left(-1\right)\,
 b}\over{3}} , x=\left({{\sqrt{27\,d^2+\left(4\,b^3-18\,b\,c\right)\,
 d+4\,c^3-b^2\,c^2}}\over{2\,3^{{{3}\over{2}}}}}+{{b\,c-3\,d}\over{6
 }}+{{\left(-1\right)\,b^3}\over{27}}\right)^{{{1}\over{3}}}-{{{{c
 }\over{3}}+{{\left(-1\right)\,b^2}\over{9}}}\over{\left({{\sqrt{27\,
 d^2+\left(4\,b^3-18\,b\,c\right)\,d+4\,c^3-b^2\,c^2}}\over{2\,3^{{{3
 }\over{2}}}}}+{{b\,c-3\,d}\over{6}}+{{\left(-1\right)\,b^3}\over{27
 }}\right)^{{{1}\over{3}}}}}+{{\left(-1\right)\,b}\over{3}} \right] 
 $$
 
I doubt that anybody has ever figured out how to alter these solutions to make them safe for numerical evalution. If you need to solve a cubic equation numerically, the exact representations for the solutions is a _terrible_ option.
